# 对象引用、可变性和垃圾回收

In [ ]:
a = [1, 2, 3]
b = a # a和b引用同一个列表，而不是列表副本

In [1]:
# is 比较变量的标识，== 比较变量的值
# a == b 等同于 a.__eq__(b)
a = None
print (a == None)
print (a is None)

True
True


In [2]:
t1 = (1,2,[30,40])
t2 = (1,2,[30,40])
print (t1 == t2)
print (t1 is t2)

True
False


In [3]:
t1[-1].append(99)
print (t1)
print (t1 == t2)
print (t1 is t2)

(1, 2, [30, 40, 99])
False
False


In [6]:
# 默认做浅复制
l1 = [3, [55, 44], (7,8,9)]
l2 = list(l1)
print (l1 == l2)
print (l1 is l2)
print (l1[1] is l2[1]) # 默认只复制了最外层容器

True
False
True


In [30]:
# 为任意对象做浅复制和深复制
# copy() 和 deepcopy()
class Bus:
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [31]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1) # 浅复制
bus3 = copy.deepcopy(bus1) # 深复制

In [32]:
print (id(bus1), id(bus2), id(bus3))

4619225632 4619225744 4619225912


In [34]:
print (id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))

4618669704 4618669704 4618546568


In [33]:
bus1.drop('Bill')
print (bus1.passengers)
print (bus2.passengers)

['Alice', 'Claire', 'David']
['Alice', 'Claire', 'David']


In [35]:
# 循环引用
a = [10, 20]
b = [a, 30]
a.append(b)

In [38]:
print (a)

[10, 20, [[...], 30]]


In [43]:
# python 唯一支持的参数传递就是共享传参
def test(a):
    print (id(a))
    a = a + 1
    print (id(a))
a = 1
print (id(a))
test(a)

4564604320
4564604320
4564604352


In [44]:
def f(a, b):
    a += b
    return a
x = 1
y = 2
print (f(x,y))
a = [1, 2]
b = [3, 4]
print (f(a, b))
print (a, b)
t = (10, 20)
u = (30, 40)
print (f(t, u))
print (t, u)

3
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)
(10, 20) (30, 40)


In [45]:
# !!!!!!!!!! 不要使用可变类型作为参数的默认值 !!!!!!!!!!!
class HauntedBus:
    """备受幽灵乘客折磨的校车"""
    def __init__(self, passengers = []):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [46]:
bus1 = HauntedBus(['Alice', 'Bill'])
print (bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print (bus1.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']


In [47]:
bus2 = HauntedBus()
bus2.pick('Carrie')
print (bus2.passengers)
bus3 = HauntedBus()
print (bus3.passengers)

['Carrie']
['Carrie']


In [48]:
bus3.pick('Dave')
print (bus2.passengers)
print (bus2.passengers is bus3.passengers)

['Carrie', 'Dave']
True


In [49]:
# 对象绝对不会自行销毁；然而，无法得到对象时，可能会被当垃圾回收
# del 语句删除名称，而不是对象。del命令可能会导致对象被当做垃圾回收，但当且仅当删除的变量保存的是对象最后一个引用，或无法得到对象时。
# 重新绑定也可能会导致对象的引用数量归零，导致对象销毁。

In [1]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye ():
    print ('Gone with the wind...')
ender = weakref.finalize(s1, bye)
print (ender.alive)
del s1
print (ender.alive)
s2 = 'spam'
print (ender.alive)

True
True
Gone with the wind...
False


In [2]:
# 弱引用
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
print (wref())
a_set = {2, 3, 4}
print (wref())

{0, 1}
None


In [3]:
# WeakValueDictionary 经常用于缓存
class Cheese:
    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

print (sorted(stock.keys()))
del catalog
print (sorted(stock.keys()))
del cheese
print (sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]
